# Intro to ML in R Bootcamp

<img src="modeling_process.jpg",width=800,height=400 >
[from: https://www.slideshare.net/CharlesVestur/building-a-performing-machine-learning-model-from-a-to-z]



# R Packages We'll Use

#### Exploratory Data Analysis + Feature Engineering
* tidyverse: set of packages for tidy data science (https://www.tidyverse.org)
* visdat: rOpenSci package for visualizing dataframes (https://github.com/ropensci/visdat)
* dummies: package for creating dummy variables (https://cran.r-project.org/web/packages/dummies/dummies.pdf)

#### Building Regression Models
* caret: package for Classification And REgression Training (https://topepo.github.io/caret/index.html)
* AppliedPredictiveModeling: companion package to the book of the same name (http://appliedpredictivemodeling.com)
* ROCR: package for visualizing performance of scoring classifiers (https://cran.r-project.org/web/packages/ROCR/ROCR.pdf)

#### Building Decision Trees + Random Forests
* rpart: package for Recursive Partioning And Regression Trees (https://cran.r-project.org/web/packages/rpart/rpart.pdf)
* rpart.plot: package for plotting rpart trees (https://cran.r-project.org/web/packages/rpart.plot/rpart.plot.pdf)
* rattle: package providing graphical interface for using R -- we're using to plot pretty trees (https://rattle.togaware.com)
* RColorBrewer: package for creating nice color palettes (https://cran.r-project.org/web/packages/RColorBrewer/RColorBrewer.pdf)
* randomForest: package for creating random forests (https://cran.r-project.org/web/packages/randomForest/randomForest.pdf)

#### Unsupervised Learning
* dbscan: package for doing density-based clustering (https://cran.r-project.org/web/packages/dbscan/dbscan.pdf)
* factoextra: ggplot2-based package for extracting and visualizing multivariate analyses (https://cran.r-project.org/web/packages/factoextra/index.html)

#### Deep Learning / Neural Nets
* keras: interface for R into the keras deep learning library (https://keras.rstudio.com)




### To install all of the above, open an R session and run:

In [1]:
install.packages(c("tidyverse", "caret", "AppliedPredictiveModeling", "keras", "modelr", "rpart",
                 "visdat", "dummies", "rpart.plot", "rattle", "RColorBrewer", "randomForest", 
                 "dbscan", "factoextra", "ROCR"))

Installing packages into ‘/Users/caitlin/Library/R/3.5/library’
(as ‘lib’ is unspecified)



  There is a binary version available but the source version is later:
           binary source needs_compilation
rpart.plot  3.0.1  3.0.2             FALSE


The downloaded binary packages are in
	/var/folders/y2/1hpzb1qn5j9fkdq211q5q5j00000gn/T//RtmppYeZBs/downloaded_packages


installing the source packages ‘rpart.plot’, ‘rattle’



# Sources

* Alligator data: https://www.r-bloggers.com/simple-linear-regression-2/
* Abalone data: http://archive.ics.uci.edu/ml/datasets/Abalone
* Chris Albon's machine learning flashcards: https://machinelearningflashcards.com
    * $12 for 300; highly recommended (by me)


# Resources

* DataCamp’s Machine Learning with R (4 course track): https://www.datacamp.com/tracks/machine-learning
* Modeling in the tidyverse: http://r4ds.had.co.nz/model-basics.html
* Exploratory visualizations: https://machinelearningmastery.com/data-visualization-in-r/
* Useful worked linear regression example: https://cfss.uchicago.edu/persp003_linear_regression.html
* More useful worked examples: https://tutorials.iq.harvard.edu/R/Rstatistics/Rstatistics.html
* MLR is working towards a scikit-like implementation https://github.com/mlr-org/mlr
* Kaggle tutorials and notebooks: http://www.kaggle.com
